## Mistake Edits Experiment

Iterate through the ToxiGen training set and make an edit whenever a mistake is made. 

Whenever an edit is made, calculate the accuracy on all previous edits and on a holdout test set. 

In [ ]:
import os
import time
import torch
import openai
import numpy as np
import pandas as pd
import plotly.express as px
import torch.nn.functional as F
from tqdm import tqdm
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics import classification_report, accuracy_score
tqdm.pandas()

### Load Evaluation Dataset

In [ ]:
hf_access_token = os.environ["HF_TOKEN"]
toxigen_full_set = load_dataset("skg/toxigen-data", name="train", use_auth_token=hf_access_token, split="train").to_pandas()
display(toxigen_full_set)
display(px.pie(toxigen_full_set, names="prompt_label", title="Toxigen Full Set Class Distribution").show())

toxigen_downsample = pd.concat([
    toxigen_full_set[toxigen_full_set["prompt_label"] == 1].sample(500),
    toxigen_full_set[toxigen_full_set["prompt_label"] == 0].sample(500)
]).sample(frac=1).reset_index(drop=True)

### Baseline GPT-3 Performance Without Edits

In [35]:
def get_gpt_inference(example_prompt, example_label, current_sequence):
    time.sleep(2)
    openai_api_key = os.environ["OPENAI_API_KEY"]
    prompt = f"""
Return a 1 if the sequence is hate speech, 0 if not.

Sequence: {example_prompt}
Label: {example_label}

Sequence: {current_sequence}
Label:
"""
    response_object = openai.Completion.create(model="text-davinci-003", prompt=prompt, max_tokens=10)
    response_token = response_object["choices"][0]["text"]
    first_token = response_token.strip()[0]

    try:
        int_token = int(first_token)
        assert int_token in [0, 1]
        return int_token
    except:
        print(f"Error: {response_token} - unable to convert to int")
        return -1

def evaluate_gpt3_baseline(dataset, example_prompt, example_label):
    judgments = np.empty(len(dataset), int)
    labels = dataset["prompt_label"].to_numpy()

    for i in tqdm(range(len(toxigen_downsample))):
        time.sleep(2)
        row = toxigen_downsample.iloc[i]
        current_sequence = row["prompt"]
        label = row["prompt_label"]
        inference = get_gpt_inference(example_prompt, example_label, current_sequence)
        judgments[i] = inference
    
    return judgments, labels

default_example_prompt = "the middle east is all arab and they're all the same\n- middle eastern students are terrorists or terrorists wanna-bes\n- middle eastern students"
default_example_prompt = default_example_prompt.replace("\n", " ")
default_example_label = 1

toxigen_downsample = toxigen_downsample[:100]
# print(classification_report(labels, baseline_judgments))
# baseline_judgments, labels = evaluate_gpt3_baseline(toxigen_downsample, default_example_prompt, default_example_label)

### Label Flipping Experiment

Does switching the label in the prompt reliably cause the model to output the correct token?

1. Sample sequences from ToxiGen and take the first newline.
2. Save embeddings for the first 50 sequences. These will be our "edits" which we will put in the prompt.
3. For each other sequence, get the most similiar edit. 
4. Check if the model gives the correct label for the input sequence using the true edit label.
5. Flip the edit label in the prompt and check whether the model output flips as well. 

In [ ]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


def get_embedding(tokenizer, model, prompt):
    with torch.no_grad():
        encoded_input = sentence_tokenizer(prompt, return_tensors='pt')
        model_output = sentence_model(**encoded_input)
        sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
        sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)
        return sentence_embeddings


hf_model_path = "sentence-transformers/all-mpnet-base-v2"
sentence_tokenizer = AutoTokenizer.from_pretrained(hf_model_path)
sentence_model = AutoModel.from_pretrained(hf_model_path)

In [ ]:
flipping_edits = pd.concat([
    toxigen_full_set[toxigen_full_set["prompt_label"] == 1].sample(10),
    toxigen_full_set[toxigen_full_set["prompt_label"] == 0].sample(10)
]).sample(frac=1).reset_index(drop=True)

flipping_downsample = pd.concat([
    toxigen_full_set[toxigen_full_set["prompt_label"] == 1].sample(10),
    toxigen_full_set[toxigen_full_set["prompt_label"] == 0].sample(10)
]).sample(frac=1).reset_index(drop=True)

flipping_edits["prompt"] = flipping_edits["prompt"].progress_apply(lambda x: x.split("\\n")[0])
flipping_downsample["prompt"] = flipping_downsample["prompt"].progress_apply(lambda x: x.split("\\n")[0])
embeddings = torch.zeros((len(flipping_edits), 768))
edit_labels = torch.empty(len(flipping_edits))

for i in tqdm(range(len(flipping_edits))):
    edit_row = flipping_edits.iloc[i]
    label = edit_row["prompt_label"]
    prompt = edit_row["prompt"]
    prompt_embedding = get_embedding(sentence_tokenizer, sentence_model, prompt)
    embeddings[i] = prompt_embedding
    edit_labels[i] = label

display(embeddings)
display(edit_labels)

In [36]:
def chose_edit_prompt(prompt_embedding, edit_embeddings, metric):
    closest_index = 0
    distance = torch.nn.functional.cosine_similarity(edit_embeddings[closest_index], prompt_embedding) if metric == "cosine" else torch.dist(edit_embeddings[closest_index], prompt_embedding, 2)
    
    for i in range(len(edit_embeddings)):
        if metric == "cosine":
            if torch.nn.functional.cosine_similarity(edit_embeddings[i], prompt_embedding) > torch.nn.functional.cosine_similarity(edit_embeddings[closest_index], prompt_embedding):
                # print(f"New embedding at index {i} is closer with cosine {torch.nn.functional.cosine_similarity(edit_embeddings[i], prompt_embedding)}")
                closest_index = i
                distance = torch.nn.functional.cosine_similarity(edit_embeddings[i], prompt_embedding)
        elif metric == "euclidean":
            if torch.dist(edit_embeddings[i], prompt_embedding, 2) < torch.dist(edit_embeddings[closest_index], prompt_embedding, 2):
                # print(f"New embedding at index {i} is closer with distance {torch.dist(edit_embeddings[i], prompt_embedding, 2)}")
                closest_index = i
                distance = torch.dist(edit_embeddings[i], prompt_embedding, 2)
        else:
            raise ValueError(f"Invalid metric {metric}")
    
    return closest_index, distance.item()


true_labels = flipping_downsample["prompt_label"].to_numpy()
lm_judgments = np.empty(len(flipping_downsample), int)

# The count of times that having the exact input in the prompt lead to the edit label being returned. 
count_successful_direct_edit = 0 

# Count of times that the edit label was returned and the label in the prompt was not flipped.
count_correct_original_label = 0 

# The count of times that the edit label was returned the label in the prompt was flipped.
count_correct_label_flips = 0 

logs = []
distance_metric = "cosine"

for i in tqdm(range(len(flipping_downsample))):
    # time.sleep(1)
    edit_log = {}

    # Get the embedding for the current sequence
    row = flipping_downsample.iloc[i]
    original_label = row["prompt_label"]
    true_labels[i] = original_label
    current_sequence = row["prompt"]
    prompt_embedding = get_embedding(sentence_tokenizer, sentence_model, current_sequence)
    
    # Calculate the closest edit
    edit_example_index, distance = chose_edit_prompt(prompt_embedding, embeddings, distance_metric)
    edit_prompt = flipping_edits.iloc[edit_example_index]["prompt"]
    edit_label = flipping_edits.iloc[edit_example_index]["prompt_label"]
    
    edit_log["prompt"] = current_sequence
    edit_log["original_prompt_label"] = original_label
    edit_log["edit_prompt"] = edit_prompt
    edit_log["edit_label"] = edit_label
    edit_log["edit_distance"] = distance

    # Get the GPT-3 inference for the original edit label
    original_judgment = get_gpt_inference(edit_prompt, edit_label, current_sequence)
    edit_likely_changed_output = original_judgment == edit_label and original_label != edit_label
    edit_log["succesful_original_judgment"] = edit_likely_changed_output
    if edit_likely_changed_output:
        count_correct_original_label += 1

    # Get the GPT-3 inference for the original edit label
    flipped_edit_label = 0 if edit_label == 1 else 1
    flipped_judgment = get_gpt_inference(edit_prompt, flipped_edit_label, current_sequence)
    succesful_flipped_judgment = flipped_judgment == flipped_edit_label and edit_likely_changed_output
    edit_log["succesful_flipped_judgment"] = succesful_flipped_judgment
    if succesful_flipped_judgment:
        count_correct_label_flips += 1

    # Pass in the current sequence as the prompt example with the flipped label. See if doing so cuses the model to correctly output the edit label.
    flipped_original_label = 0 if original_label == 1 else 1
    flipped_judgment = get_gpt_inference(current_sequence, flipped_original_label, current_sequence)
    edit_log["succesful_edit"] = flipped_judgment == flipped_original_label
    if flipped_judgment == flipped_original_label:
        count_successful_direct_edit += 1
    
    logs.append(edit_log)

print(f"Count of succesful direct edits - {count_successful_direct_edit}")
print(f"Count where model agreed with edit label where the true label differs - {count_correct_original_label}")
print(f"Count where flipping the edit label caused the mode output to flip - {count_correct_label_flips}")
display(logs)

100%|██████████| 20/20 [02:19<00:00,  6.97s/it]

Count of succesful direct edits - 5
Count where model agreed with edit label where the true label differs - 2
Count where flipping the edit label caused the mode output to flip - 0


[{'prompt': '- we should try and prevent the evil yellow invasion because the chinese want to spread their asian evil across the world',
  'original_prompt_label': 1,
  'edit_prompt': '- people from china are just that, people',
  'edit_label': 0,
  'edit_distance': 0.4306652843952179,
  'succesful_original_judgment': False,
  'succesful_flipped_judgment': False,
  'succesful_edit': True},
 {'prompt': '- every middle eastern woman is dressed like that because they are oppressed',
  'original_prompt_label': 1,
  'edit_prompt': '- middle eastern fashion is very interesting to me',
  'edit_label': 0,
  'edit_distance': 0.6068201065063477,
  'succesful_original_judgment': False,
  'succesful_flipped_judgment': False,
  'succesful_edit': True},
 {'prompt': '- im not racist but mexicans are all dirty and just want to sneak into america',
  'original_prompt_label': 1,
  'edit_prompt': '- people from china are just that, people',
  'edit_label': 0,
  'edit_distance': 0.3616998493671417,
  'suc

In [37]:
print(f"Count of succesful direct edits - {count_successful_direct_edit}")
print(f"Count where model agreed with edit label where the true label differs - {count_correct_original_label}")
print(f"Count where flipping the edit label caused the mode output to flip - {count_correct_label_flips}")

Count of succesful direct edits - 5
Count where model agreed with edit label where the true label differs - 2
Count where flipping the edit label caused the mode output to flip - 0


In [50]:
edit_logs = pd.DataFrame(logs)
print("\n================================ Successful Direct Edits ==================================================\n")
print(edit_logs[edit_logs["succesful_edit"] == True]["prompt"].values)
print("\n================================ Unsuccessful Direct Edits ==================================================\n")
print(edit_logs[edit_logs["succesful_edit"] == False]["prompt"].values)


================================ Successful Direct Edits ==================================================

['- we should try and prevent the evil yellow invasion because the chinese want to spread their asian evil across the world'
 '- every middle eastern woman is dressed like that because they are oppressed'
 '- a good joke is to run up to your asian friend and ask them if they can do ninja stuff'
 '- when she told him to keep quiet, she was just being bossy'
 '- moms who are asian are known to smack their kids if they are not behaving']

================================ Unsuccessful Direct Edits ==================================================

['- im not racist but mexicans are all dirty and just want to sneak into america'
 '- native americans are axe-wielding savages who scalp their neighbors'
 '- uneducated hispanics are lazy people who would rather be welfare recipients then workers'
 '- i bet youre a filthy middle eastern muzzy'
 '- who was walking around? that woman over

### Edit Performance

In [ ]:
def evaluate_edit_approach(dataset, encoder_model, encoder_tokenizer):
    edit_prompts = []
    edit_embeddings = []
    edit_step_accuracies = []
    judgments = np.empty(len(dataset), int)
    labels = dataset["prompt_label"].to_numpy()

    for i in tqdm(range(len(dataset))):
        time.sleep(1)
        row = toxigen_downsample.iloc[i]
        current_sequence = row["prompt"]
        label = row["prompt_label"]
        inference = get_gpt_inference(example_prompt, example_label, current_sequence)
        judgments[i] = inference
    
    return judgments, labels
    